# 策略说明

    开仓条件：
        1. 股价沿5日均线上线
        2. 不存在前一周期rsi超标、当前周期rsi超标以及上涨3%后rsi超标情况。rsi指标处于90%高位为超标
    平仓条件：
        1. rsi 30分钟线高位,或
        2. -5%止损，或
        3. 30分钟两次上攻失败,或
        4. 5天内两次上攻失败

In [ ]:
from alpha.notebook import *
from alpha.strategies.z05 import Z05
await init_notebook(use_omicron=True)

In [ ]:
async def scan(dt, n):
    end = tf.day_shift(arrow.get(dt), 0)
    nbars = n + 5 - 1
    start = tf.day_shift(end, -nbars + 1)
    
    results = []
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)
            msr = maline_support_ratio(bars["close"], 5, n)
            br = bullish_candlestick_ratio(bars, n)
            bd = real_body(bars)
            d1 = ma_d1(bars["close"], 5)[-1]
            d2 = ma_d2(bars["close"], 5)[-1]
            results.append([sec.display_name, code, msr, br, bd, d1, d2])
        except Exception as e:
            print(e)
            continue
            
    return results
            
results = await scan("2021-10-26", 7)
say("finished")

In [ ]:
df = pd.DataFrame(results, columns=["name", "code", "msr", "br", "body", "d1", "d2"])

In [ ]:
df.nlargest(20, ["msr", "br", "d2", "d1"])[df.d2>0]

# Z05

## 持仓3天强平

In [ ]:
z05 = Z05(holding_days=3)
summary = await z05.backtest("2021-10-21", "2021-10-21")

In [3]:
summary

{'start': datetime.date(2021, 10, 21),
 'end': datetime.date(2021, 10, 21),
 'duration': 0,
 'returns': 0.15186554193496704,
 'avg_gains': 0.015186554193496704,
 'best_trade': 0.09061837196350098,
 'worst_trade': -0.05401754379272461,
 'win_rate': 0.7,
 'exposure_time': 41,
 'total_trades': 10}

In [4]:
z05.trades

[{'code': '000803.XSHE',
  'buy_price': 18.15,
  'sell_price': 18.26,
  'gains': 0.006060600280761719,
  'order_date': datetime.date(2021, 10, 21),
  'sell_at': datetime.datetime(2021, 10, 26, 15, 0, tzinfo=tzfile('/usr/share/zoneinfo/Asia/Shanghai')),
  'close_type': 'expired',
  'params': {'msr': 1.0,
   'bcr': 0.8571428571428571,
   'rb': 0.2019866704940796,
   'd1': 0.0190160870552063,
   'd2': -0.01857947111129761,
   'prsi': 0.8469516180291302,
   'tvd1': 1.4596663445690037,
   'tvd2': 1.9315761254358066,
   'tvd3': 1.282865770207034,
   'vr_mean': -0.39751727061490605}},
 {'code': '002812.XSHE',
  'buy_price': 295.89,
  'sell_price': 287.92,
  'gains': -0.026935696601867676,
  'order_date': datetime.date(2021, 10, 21),
  'sell_at': datetime.datetime(2021, 10, 26, 15, 0, tzinfo=tzfile('/usr/share/zoneinfo/Asia/Shanghai')),
  'close_type': 'expired',
  'params': {'msr': 1.0,
   'bcr': 0.8571428571428571,
   'rb': 0.19793522357940674,
   'd1': 0.023077750205993654,
   'd2': -0.0089

## 持仓4天强平

In [ ]:
z05 = Z05(holding_days=4)
summary = await z05.backtest("2021-10-21", "2021-10-21")

In [6]:
summary

{'start': datetime.date(2021, 10, 21),
 'end': datetime.date(2021, 10, 21),
 'duration': 0,
 'returns': 0.4061797261238098,
 'avg_gains': 0.04061797261238098,
 'best_trade': 0.1940298080444336,
 'worst_trade': -0.0865098237991333,
 'win_rate': 0.7,
 'exposure_time': 47,
 'total_trades': 10}

In [7]:
z05.trades

[{'code': '000803.XSHE',
  'buy_price': 18.15,
  'sell_price': 19.84,
  'gains': 0.09311294555664062,
  'order_date': datetime.date(2021, 10, 21),
  'sell_at': datetime.datetime(2021, 10, 27, 11, 30, tzinfo=tzfile('/usr/share/zoneinfo/Asia/Shanghai')),
  'close_type': 'rsi',
  'params': {'msr': 1.0,
   'bcr': 0.8571428571428571,
   'rb': 0.2019866704940796,
   'd1': 0.0190160870552063,
   'd2': -0.01857947111129761,
   'prsi': 0.8469516180291302,
   'tvd1': 1.4596663445690037,
   'tvd2': 1.9315761254358066,
   'tvd3': 1.282865770207034,
   'vr_mean': -0.39751727061490605,
   'sell_rsi': 91.77580598307314,
   'sell_prsi': 0.9905041326292927}},
 {'code': '002812.XSHE',
  'buy_price': 295.89,
  'sell_price': 297.0,
  'gains': 0.003751397132873535,
  'order_date': datetime.date(2021, 10, 21),
  'sell_at': datetime.datetime(2021, 10, 27, 15, 0, tzinfo=tzfile('/usr/share/zoneinfo/Asia/Shanghai')),
  'close_type': 'expired',
  'params': {'msr': 1.0,
   'bcr': 0.8571428571428571,
   'rb': 0.19

# 昨天的信号

In [8]:
z05 = Z05(holding_days=3)

end = arrow.get("2021-10-27").date()
await z05.scan(end)

2021-10-28 09:18:55,076 E 89954 alpha.strategies.z05:__init__:67 | [Errno 2] No such file or directory: '/Users/aaronyang/zillionare/aplha/z05.model.pkl'
Traceback (most recent call last):
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/alpha/strategies/z05.py", line 65, in __init__
    self.model = self.load_model(model_file)
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/alpha/strategies/z05.py", line 71, in load_model
    with open(model_file, "rb") as f:
FileNotFoundError: [Errno 2] No such file or directory: '/Users/aaronyang/zillionare/aplha/z05.model.pkl'


In [10]:
z05.long_orders

[{'code': '002182.XSHE',
  'order_date': datetime.date(2021, 10, 27),
  'buy_price': 21.36,
  'params': {'msr': 1.0,
   'bcr': 0.8571428571428571,
   'rb': 0.18666672706604004,
   'd1': 0.013218462467193604,
   'd2': -0.0307769775390625,
   'prsi': 0.7863641814484731,
   'tvd1': 1.137357645532997,
   'tvd2': 1.0759899494831633,
   'tvd3': -1.1792753920927437,
   'vr_mean': 0.048485875472023376}},
 {'code': '002312.XSHE',
  'order_date': datetime.date(2021, 10, 27),
  'buy_price': 16.96,
  'params': {'msr': 1.0,
   'bcr': 0.8571428571428571,
   'rb': 0.17127060890197754,
   'd1': 0.03155255317687989,
   'd2': 0.0038056135177612326,
   'prsi': 0.8980408443739177,
   'tvd1': 1.2884269091031544,
   'tvd2': 1.638328332077182,
   'tvd3': 1.2039305165597034,
   'vr_mean': -0.17433263069365867}},
 {'code': '002375.XSHE',
  'order_date': datetime.date(2021, 10, 27),
  'buy_price': 7.5999994,
  'params': {'msr': 1.0,
   'bcr': 0.8571428571428571,
   'rb': 0.10144913196563721,
   'd1': 0.01507381